# 2022-11-14 Ratel Hands-on

## Last time

* Intro to solid mechanics, Ratel
* Singularities and $hp$ adaptivity
* Cost of sparse matrices
* GPU performance with Ratel and context

## Today

* Solver diagnostics
* Reading profiles
* Amortizing costs

<video src="img/schwarz-pendulum.webm" autoplay loop controls width="100%" />

In [1]:
using Plots
default(linewidth=3)
using LinearAlgebra
using SparseArrays

# Running on Alpine

```console
$ ssh login.rc.colorado.edu
rc$ module load slurm/alpine
rc$ acompile
acompile$ . /projects/jeka2967/activate.bash
```

```console
$ git clone \
    https://gitlab.com/micromorph/ratel
$ cd ratel/examples
$ mpiexec -n 1 ratel-quasistatic \
    -options_file FILE.yml
```

## Running in Docker

Clone the Ratel repository and cd into it
```console
host$ docker run -it --rm -v $(pwd):/work registry.gitlab.com/micromorph/ratel
$ mpiexec -n 2 ratel-quasistatic -options_file ex02-quasistatic-elasticity-multi-material.yml
```

# Suggested test problem

```console
$ mpiexec -n 6 ratel-quasistatic -options_file ex02-quasistatic-elasticity-schwarz-pendulum.yml
```

# Quasi-Newton methods (BFGS)

BFGS is a method that does not require a linear solve.
* It starts with an operation $J_0^{-1}$, which is meant to be an approximation of the inverse Jacobian (like a preconditioner).
* Each iteration creates a symmetric rank-2 update,
$$J_{k+1}^{-1} = \left(I - \frac{s_k z_k^T}{s_k^T z_k}\right)                                                                                                           J_k^{-1} \left(I - \frac{s_k z_k^T}{s_k^T z_k}\right) + \frac{s_k s_k^T}{s_k^T z_k}$$
* BFGS is equivalent to conjugate gradients for a linear problem.

* `-snes_type qn -snes_qn_scale_type jacobian`
  * This uses an iterative solve for $J_0$
* Add `-ksp_type preonly` (skip the iterative solve)

# Solids: efficient matrix-free Jacobians
## cf. [Davydov et al. (2020)](https://doi.org/10.1002/nme.6336)

<img src="img/libceed-solids-initial-current.png" width="80%" />
<img src="img/libceed-solids-jacobian-table.png" width="80%" />

# Suggestions (use `-ts_view` and `-log_view`)

* Change from initial to current configuration

```console
$ git grep model: examples/*.yml
```

* Try assembling and using AMG directly (instead of p-MG first): `-multigrid amg_only`
  * And try a direct solve: `-pc_type cholesky`

* How does thickness affect solve cost? Edit the input file or override from command line.

* Visualization (if you have Paraview)
  * `-ts_monitor_solution cgns:sol.cgns`
  * `-view_diagnostic_quantities vtk:diagnostic.vtu`